In [1]:

import numpy as np
import scipy.signal as signal
import msicpe.san as san 

def charger_signal(chemin_fichier: str):
    """
    Charge un signal stocké dans un fichier .npz contenant :
      - 't' : vecteur temps (en secondes)
      - 's' : signal temporel

    Retourne :
      - temps : vecteur temps
      - signal_temporel : signal
      - fe : fréquence d'échantillonnage (en Hz)
    """
    donnees = np.load(chemin_fichier)
    temps = donnees["t"]
    signal_temporel = donnees["s"]
    fe = 1.0 / (temps[1] - temps[0])
    return temps, signal_temporel, fe


def centrer_signal(signal_temporel: np.ndarray) -> np.ndarray:
    """Supprime la composante continue du signal (centrage)."""
    return signal_temporel - np.mean(signal_temporel)


def calculer_spectre_amplitude(signal_centre: np.ndarray, fe: float):
    """
    Calcule le spectre d'amplitude du signal centré.

    Étapes :
      - application d'une fenêtre de Hann (scipy.signal)
      - calcul de la FFT (np.fft)
      - calcul du module du spectre

    Retourne :
      - frequences : axe des fréquences (Hz)
      - amplitude_spectre : |S(f)|
    """
    nb_echantillons = len(signal_centre)

    # Fenêtre de Hann pour utiliser scipy.signal
    fenetre = signal.windows.hann(nb_echantillons)
    signal_fenetre = signal_centre * fenetre

    spectre_complexe = np.fft.rfft(signal_fenetre)
    frequences = np.fft.rfftfreq(nb_echantillons, d=1.0/fe)
    amplitude_spectre = np.abs(spectre_complexe)

    return frequences, amplitude_spectre


def normaliser_spectre_amplitude(amplitude_spectre: np.ndarray) -> np.ndarray:
    """
    Normalise le spectre d'amplitude de sorte que l'amplitude maximale soit égale à 1.

    Cette normalisation doit être appliquée AVANT l'appel à detecter_fondamentale,
    conformément aux consignes du TP.
    """
    max_amp = np.max(amplitude_spectre)
    if max_amp == 0:
        return amplitude_spectre
    return amplitude_spectre / max_amp


def detecter_fondamentale(frequences: np.ndarray,
                          amplitude_norm: np.ndarray,
                          fmin: float = 50.0,
                          fmax: float = 500.0) -> float:
    """
    Détecte la fréquence fondamentale du signal en cherchant le maximum du spectre
    normalisé dans la bande [fmin, fmax].

    Entrées :
      - frequences : vecteur des fréquences (Hz)
      - amplitude_norm : spectre normalisé (max = 1)
      - fmin, fmax : borne min et max de recherche de f0

    Sortie :
      - f0 : fréquence fondamentale estimée (Hz)
    """
    masque = (frequences >= fmin) & (frequences <= fmax)
    if not np.any(masque):
        raise ValueError("Aucune fréquence dans la bande spécifiée pour la détection de f0.")
    indice_pique = np.argmax(amplitude_norm[masque])
    f0 = frequences[masque][indice_pique]
    return f0


def extraire_harmoniques(frequences: np.ndarray,
                         amplitude_norm: np.ndarray,
                         f0: float,
                         nombre_harmoniques: int = 5):
    """
    Extrait les 'nombre_harmoniques' premières harmoniques du signal :

      - fréquences harmoniques : k * f0
      - amplitudes associées : valeur du spectre normalisé au bin le plus proche

    Retourne :
      - freq_harmoniques : tableau des fréquences harmoniques (Hz)
      - amp_harmoniques : tableau des amplitudes normalisées associées
    """
    freq_harmoniques = f0 * np.arange(1, nombre_harmoniques + 1)
    amp_harmoniques = []

    for f_h in freq_harmoniques:
        indice_proche = np.argmin(np.abs(frequences - f_h))
        amp_harmoniques.append(amplitude_norm[indice_proche])

    return freq_harmoniques, np.array(amp_harmoniques)


def normaliser_vecteur_harmonique(amplitudes: np.ndarray) -> np.ndarray:
    """
    Normalise un vecteur d'amplitudes harmoniques par la somme de ses composantes.

    Si la somme est nulle, le vecteur est renvoyé tel quel.
    """
    somme = np.sum(amplitudes)
    if somme == 0:
        return amplitudes
    return amplitudes / somme